# Ejemplo Práctico 02

<img src="https://images.indianexpress.com/2021/04/hh.jpg" align="center"  width="360" height="240"/>


Esta semana revisaremos datos del **Índice de Libertad de Prensa** que confecciona cada año la asociación de Reporteros Sin Fronteras.


## Diccionario de datos


|Variable       |Clase               |Descripción |
|:--------------|:-------------------|:-----------|
| codigo_iso | caracter | Código ISO del país |
| pais | caracter | País |
| anio | entero | Año del resultado |
| indice | entero | Puntaje Índice Libertad de Prensa (menor puntaje = mayor libertad de prensa) |
| ranking | entero | Ranking Libertad de Prensa |


## Fuente original y adaptación
Los datos fueron extraídos de [The World Bank](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019). La fuente original es [Reporteros sin Fronteras](https://www.rsf-es.org/). 

Por otro lado, estos archivos han sido modificado intencionalmente para ocupar todo lo aprendido en clases. A continuación, una breve descripción de cada uno de los data frames:

* **libertad_prensa_codigo.csv**: contiene la información codigo_iso/pais. Existe un código que tiene dos valores.
* **libertad_prensa_anio.csv**: contiene la información pais/anio/indice/ranking. Los nombres de las columnas estan en mayúscula.




In [8]:
import numpy as np 
import pandas as pd

from os import listdir
from os.path import isfile, join

In [9]:
path = "data/pf_index/"

archivos_anio = [path + f for f in listdir(path) if 'libertad_prensa_codigo' not in f ] 
df_codigos = pd.read_csv(path + 'libertad_prensa_codigo.csv')
df_codigos.head()

,codigo_iso,pais
0,AFG,Afghanistán
1,AGO,Angola
2,ALB,Albania
3,AND,Andorra
4,ARE,Emiratos Árabes Unidos


El objetivo es tratar de obtener la mayor información posible de este conjunto de datos. Para cumplir este objetivo debe resolver las siguientes problemáticas:
 
1. Lo primero será juntar toda la información en un _solo archivo_, para ello necesitamos seguir los siguientes pasos:

 * a) Crear el archivo **df_anio**, que contenga la información de **libertad_prensa_anio.csv** para cada año. Luego, normalice el nombre de las columnas a minúscula.
 * b) Encuentre y elimine el dato que esta duplicado en el archivo **df_codigo**.
 * c) Crear el archivo **df** que junte la información del archivo **df_anio** con **df_codigo** por la columna _codigo_iso_.
 
> **Hint**: Para juntar por _anio_ ocupe la función **pd.concat**. Para juntar información por columna ocupe **pd.merge**.

In [10]:
# creamos el dataframe con la información de todos los años
anios = []
for name in archivos_anio:
    df_temp = pd.read_csv(name)
    anios.append(df_temp)

df_anio = pd.concat(anios)

# cambiamos los nombres de las columnas de df_anio a minúsculas
df_anio.columns = df_anio.columns.str.lower()

df_anio.head()

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.5,59.0
1,AGO,2001,30.2,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN


In [11]:
# eliminamos el dato repetido en df_codigos

print("Total duplicados antes")
print(df_codigos["codigo_iso"].value_counts().loc[lambda x: x>1])

df_codigos = df_codigos.drop_duplicates('codigo_iso')

print("\nTotal duplicados despues")
print(df_codigos["codigo_iso"].value_counts().loc[lambda x: x>1])

Total duplicados antes
ZWE    2
Name: codigo_iso, dtype: int64

Total duplicados despues
Series([], Name: codigo_iso, dtype: int64)


In [12]:
# juntar informacion

df = df_anio.merge(df_codigos,on = 'codigo_iso' )
df.head()

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.50,59.0,Afghanistán
1,AFG,2002,40.17,78.0,Afghanistán
2,AFG,2003,28.25,49.0,Afghanistán
3,AFG,2004,39.17,62.0,Afghanistán
4,AFG,2005,44.25,67.0,Afghanistán


2. Encontrar:
   * ¿Cuál es el número de observaciones en el conjunto de datos?   
   * ¿Cuál es el número de columnas en el conjunto de datos?   
   * Imprime el nombre de todas las columnas  
   * ¿Cuál es el tipo de datos de cada columna? 
   * Describir el conjunto de datos (**hint**: .describe())
    

In [13]:
# respuesta
(n_obs, n_cols) = df.shape
print(f"Hay {n_obs}  observaciones en el conjunto de datos")
print(f"Hay {n_cols} columnas en el conjunto de datos")

Hay 3060  observaciones en el conjunto de datos
Hay 5 columnas en el conjunto de datos


3. Desarrolle una función `resumen_df(df)` para encontrar el total de elementos distintos y vacíos por columnas.

In [14]:
# respuesta
def resumen_df(df):
    """
    funcion resumen con elementos 
    distintos y vacios por columnas
    """
    nombres = df.columns
    
    l1 = []
    l2 = []
    
    for nombre in nombres:
        pd_series = df[nombre]
        # elementos distintos 
        l_unique = pd_series.unique()
        # elementos vacios
        l_vacios = pd_series[pd_series.isna()]
        
        l1.append(len(l_unique))
        l2.append(len(l_vacios))
        
        
    
    result = pd.DataFrame({'nombres': nombres})
    result['elementos_distintos'] = l1
    result['elementos_vacios'] = l2
    
    return result

In [15]:
# retornar 
resumen_df(df)

,nombres,elementos_distintos,elementos_vacios
0,codigo_iso,180,0
1,anio,17,0
2,indice,1551,396
3,ranking,194,223
4,pais,179,0


4. Para los paises latinoamericano, encuentre por año  el país con mayor y menor `indice`.

 * a) Mediante un ciclo _for_.
 * b) Mediante un  _groupby_.

In [16]:
# respuesta

america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america = df.loc[lambda x: x.codigo_iso.isin(america)]

In [17]:
# ciclo for

dct = dict()

for pais in america:
    df_temp =df.loc[lambda x: x.codigo_iso == pais] 
    dct[pais] = (df_temp['indice'].max(),df_temp['indice'].min())
dct

{'ARG': (35826.0, 11.33),
 'ATG': (20.81, 20.81),
 'BLZ': (27.5, 17.05),
 'BOL': (35.38, 4.5),
 'BRA': (34.03, 14.5),
 'CAN': (16.53, 0.8),
 'CHL': (26.24, 6.5),
 'COL': (51.5, 35.5),
 'CRI': (14.01, 3.83),
 'CUB': (106.83, 63.81),
 'DOM': (28.34, 6.75),
 'ECU': (34.69, 5.5),
 'GRD': (12.0, 12.0),
 'GTM': (39.33, 16.5),
 'GUY': (27.21, 10.5),
 'HND': (51.13, 11.75),
 'HTI': (42.13, 15.0),
 'JAM': (12.73, 3.33),
 'MEX': (53.63, 17.67),
 'NIC': (35.53, 6.5),
 'PAN': (32.95, 9.5),
 'PER': (40.0, 9.5),
 'PRY': (35.64, 7.17),
 'SLV': (29.81, 5.75),
 'SUR': (18.2, 6.0),
 'TTO': (24.74, 1.0),
 'URY': (17.43, 4.0),
 'USA': (25.69, 4.0),
 'VEN': (49.1, 23.0)}

In [21]:
# ciclo groupby
df_america.groupby('codigo_iso')['indice'].agg({max,min})

,max,min
codigo_iso,,
ARG,35826.00,11.33
ATG,20.81,20.81
BLZ,27.50,17.05
BOL,35.38,4.50
BRA,34.03,14.50
CAN,16.53,0.80
CHL,26.24,6.50
COL,51.50,35.50
CRI,14.01,3.83


5. Para cada _país_, muestre el _indice_ máximo que alcanzo por _anio_. Para los datos nulos, rellene con el valor **0**.

**Ejemplo**:

<img src="https://raw.githubusercontent.com/vcanalesp/portafolio/9cfad4c4c98a487e7d402c3e769dc9db5dfce629/docs/projects/DataScience/03_pandas/images/img.png" alt="Girl in a jacket" >

> **Hint**: Utilice la función **pd.pivot_table**.

In [20]:
df_pivot = df.pivot_table(
    index="codigo_iso", 
    columns="anio",
    values="indice",
    fill_value=0
)
df_pivot

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
codigo_iso,,,,,,,,,,,,,,,,
AFG,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
AGO,30.2,28.00,26.50,18.00,21.50,26.50,29.50,36.50,28.50,37.80,36.50,37.84,39.89,40.42,38.35,34.96
ALB,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
AND,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63
ARE,0.0,37.00,50.25,25.75,17.50,20.25,14.50,21.50,23.75,33.49,36.03,36.73,36.73,39.39,40.86,43.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,33.00,23.84,22.02,22.32,18.80,16.41,16.69,18.25
YEM,34.8,41.83,48.00,46.25,54.00,56.67,59.00,83.38,82.13,69.22,67.26,66.36,67.07,65.80,62.23,61.66
ZAF,7.5,3.33,5.00,6.50,11.25,13.00,8.00,8.50,12.00,24.56,23.19,22.06,21.92,20.12,20.39,22.19
